In [ ]:
# Core analysis packages
import numpy as np
import os, sys
import pandas as pd
from scipy import stats
from scipy.special import comb
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import anova
# from patsy import dmatrices
import bff
import pingouin as pg
import researchpy

import networkx as nx

# Plotting packages
import matplotlib.pyplot as plt
plt.rcdefaults()
import seaborn as sns 
sns.set(style="ticks", color_codes=True)
sns.set_style("white")
sns.set_style({'xtick.bottom': True, 'ytick.left': True})
colorref = ["gray", "royalblue", "crimson", "goldenrod", "mediumorchid", "seagreen"]

# iPython magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2
%autosave 30

SMALL_SIZE = 12
MEDIUM_SIZE = 12
BIG_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIG_SIZE)  # fontsize of the figure title
cust_palette = sns.color_palette("Paired")[6:10]
cust_palette = [cust_palette[i] for i in [1,0,3,2]]

def median_split(S):
    return S > S.median()

In [ ]:
def ttest_ind(x1, x2, equivar=False, alpha=0.05, printres=False):
    n1 = len(x1)
    M1 = np.mean(x1)
    s1 = np.std(x1, ddof=1)
    n2 = len(x2)
    M2 = np.mean(x2)
    s2 = np.std(x2, ddof=1)
    
    # t-test
    [t, p] = stats.ttest_ind(x1, x2, equal_var=equivar)
    # cohen's d
    dof = n1 + n2 - 2
    sp = np.sqrt(((n1-1)*s1**2 + (n2-1)*s2**2) / dof)
    d = np.abs(M1 - M2) / sp
    # degrees of freedom
    df = (s1**2/n1 + s2**2/n2)**2 / ((s1**2/n1)**2/(n1-1) + (s2**2/n2)**2/(n2-1))
    # confidence intervals (M1 - M2) ± ts(M1 - M2)
    se = np.sqrt(sp**2/n1 + sp**2/n2)
    CI = (M1 - M2) + np.array([-1,1])*stats.t.ppf(1-alpha/2, df, loc=0, scale=1)*se

    res = (t, df, p, d, CI[0], CI[1])
    if printres:
        print("t = %.5f, df = %.5f, p = %.5f, d = %.5f, CI = (%.5f, %.5f)" % res)
    else:
        return res

In [ ]:
# Load file (from same directory as the notebook)/

df = pd.read_excel(os.path.expanduser("sensitiv.xlsx"))
DAT = pd.read_excel(os.path.expanduser("DAT.xlsx"))
df = pd.read_excel(os.path.expanduser("dyaddat.xlsx"))
df.head()

In [ ]:
# Extract columns with df.target or df["target"] or df.loc[:,"target"]
participant = df.loc[:,"ParticipantID"]

# Extract data with df.loc[:,"datastart":"dataend"]
PartAgree = df.loc[:, "PA_fact1":"PA_con4"]
SelfAgree = df.loc[:, "SA_fact1":"SA_con4"]
Change = df.loc[:, "Change_fact1":"Change_con4"]
pre = df.loc[:, "pre1":"pre22"]
# Item categories (hard-code this!)
FMC = np.array(9*["Fact"] + 9*["Myth"] + 4*["Conspiracy"])[np.newaxis, :]

sz = Change.shape
partnum = np.arange(sz[0])[:, np.newaxis]
itemnum = np.arange(sz[-1])[np.newaxis, :]

# Make all the participant X item variables the same size with numpy arrays broadcast function
names = ["SpeakAgree", "SelfAgree", "Change", "pre", "FMC", "itemnum", "partnum"]
data_packed = np.broadcast_arrays(PartAgree, SelfAgree, Change, pre, FMC, itemnum, partnum)
data_unpacked = [np.reshape(a, -1) for a in data_packed]

DATA = pd.DataFrame.from_dict({n: d for (n,d) in zip(names, data_unpacked)})

nparticipant = int(DATA["partnum"].max())+1
lmh_all = []
lmh = [0]*7 + [1]*8 + [2]*7
for p in range(nparticipant):
    idx = np.argsort(DATA.loc[DATA["partnum"]==p, "pre"])
    lmh_sorted = [x for _, x in sorted(zip(idx, lmh))]
    lmh_all += lmh_sorted
DATA.insert(5, "LMH", lmh_all)

map_dict = {0: "Low", 1: "aMod", 2: "High"}
DATA.insert(6,"lmh",DATA["LMH"].map(map_dict))

DATA.head()

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(10,4.3))

sns.regplot(x=DATA.SpeakAgree, y=DATA.Change, scatter_kws={"color": "#493D26"}, \
            line_kws={"color":"#493D26","alpha":1,"lw":3}, x_estimator=np.mean, ax=ax[0])
ax[0].set_xlim([-3.3,3.3])
ax[0].set_ylabel('Belief Change')
ax[0].set(xlabel="Partner's conversational endorsement")
ax[0].legend(['All'])
bff.panellabel(ax[0], "A")

sns.regplot(x=DATA.loc[DATA.LMH==0, "SpeakAgree"], y=DATA.loc[DATA.LMH==0, "Change"], scatter_kws={"s": 0}, \
            line_kws={"color":"#2471A3","alpha":1,"lw":2}, ax=ax[1])
sns.regplot(x=DATA.loc[DATA.LMH==1, "SpeakAgree"], y=DATA.loc[DATA.LMH==1, "Change"], scatter_kws={"s": 0}, \
            line_kws={"color":"#E9AB17","alpha":1,"lw":3}, ax=ax[1])
sns.regplot(x=DATA.loc[DATA.LMH==2, "SpeakAgree"], y=DATA.loc[DATA.LMH==2, "Change"], scatter_kws={"s": 0}, \
            line_kws={"color":"#BA4A00","alpha":1,"lw":2}, ax=ax[1])
bff.panellabel(ax[1], "B")
ax[1].set_ylim([-3,3])
ax[1].set_ylabel('')
ax[1].set(xlabel="Partner's conversational endorsement")
ax[1].legend(['Low', 'Moderate', 'High'])

sns.regplot(x=DATA.loc[DATA.FMC=="Fact", "SpeakAgree"], y=DATA.loc[DATA.FMC=="Fact", "Change"], scatter_kws={"s": 0}, \
            line_kws={"color":"#057D31","alpha":1,"lw":3}, ax=ax[2])
sns.regplot(x=DATA.loc[DATA.FMC=="Myth", "SpeakAgree"], y=DATA.loc[DATA.FMC=="Myth", "Change"], scatter_kws={"s": 0}, \
            line_kws={"color":"#E77471","alpha":1,"lw":2}, ax=ax[2])
sns.regplot(x=DATA.loc[DATA.FMC=="Conspiracy", "SpeakAgree"], y=DATA.loc[DATA.FMC=="Conspiracy", "Change"], scatter_kws={"s": 0}, \
            line_kws={"color":"#7D0552","alpha":1,"lw":2}, ax=ax[2])
ax[2].set_ylim([-3,3])
ax[2].set_ylabel('')
ax[2].set(xlabel="Partner's conversational endorsement")
ax[2].legend(['Accurate', 'Inaccurate', 'Conspiracy'])
bff.panellabel(ax[2], "C")

plt.tight_layout()
plt.savefig('figdyad.tif', dpi=900, format="tiff")

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(9,4.5), sharey=True)

sns.regplot(x=df.FOX, y=df.KNOWLEDGE_pre, scatter_kws={"s": 0}, \
            line_kws={"color":"#CD5C5C","alpha":1,"lw":2}, ax=ax[0])
sns.regplot(x=df.CNN, y=df.KNOWLEDGE_pre, scatter_kws={"s": 0}, \
            line_kws={"color":"#2E86C1","alpha":1,"lw":2}, ax=ax[0])
sns.regplot(x=df.ABC, y=df.KNOWLEDGE_pre, scatter_kws={"s": 0}, \
            line_kws={"color":"#493D26","alpha":1,"lw":2}, ax=ax[0])
sns.regplot(x=df.MSNBC, y=df.KNOWLEDGE_pre, scatter_kws={"s": 0}, \
            line_kws={"color":"#48C9B0","alpha":1,"lw":2}, ax=ax[0])
sns.regplot(x=df.NBC, y=df.KNOWLEDGE_pre, scatter_kws={"s": 0}, \
            line_kws={"color":"#8E44AD","alpha":1,"lw":2}, ax=ax[0])
sns.regplot(x=df.CBS, y=df.KNOWLEDGE_pre, scatter_kws={"s": 0}, \
            line_kws={"color":"#E6B0AA","alpha":1,"lw":2}, ax=ax[0])
sns.regplot(x=df.PBS, y=df.KNOWLEDGE_pre, scatter_kws={"s": 0}, \
            line_kws={"color":"#935116","alpha":1,"lw":2}, ax=ax[0])


#ax.set_xlim([-1.5,1.5])
ax[0].set_ylabel('Knowledge')
ax[0].set(xlabel='News Media Conspumption')
#ax[0].set_title("FOX and CNN", fontsize=18)
#stats.pearsonr(df.Trump, df.KNOWLEDGE_pre)


sns.regplot(x=df.FOX, y=df.CONSPIRACY_pre, scatter_kws={"s": 0}, \
            line_kws={"color":"#CD5C5C","alpha":1,"lw":2}, ax=ax[1])
sns.regplot(x=df.CNN, y=df.CONSPIRACY_pre, scatter_kws={"s": 0}, \
            line_kws={"color":"#2E86C1","alpha":1,"lw":2}, ax=ax[1])
sns.regplot(x=df.ABC, y=df.CONSPIRACY_pre, scatter_kws={"s": 0}, \
            line_kws={"color":"#493D26","alpha":1,"lw":2}, ax=ax[1])
sns.regplot(x=df.MSNBC, y=df.CONSPIRACY_pre, scatter_kws={"s": 0}, \
            line_kws={"color":"#48C9B0","alpha":1,"lw":2}, ax=ax[1])
sns.regplot(x=df.NBC, y=df.CONSPIRACY_pre, scatter_kws={"s": 0}, \
            line_kws={"color":"#8E44AD","alpha":1,"lw":2}, ax=ax[1])
sns.regplot(x=df.CBS, y=df.CONSPIRACY_pre, scatter_kws={"s": 0}, \
            line_kws={"color":"#E6B0AA","alpha":1,"lw":2}, ax=ax[1])
sns.regplot(x=df.PBS, y=df.CONSPIRACY_pre, scatter_kws={"s": 0}, \
            line_kws={"color":"#935116","alpha":1,"lw":2}, ax=ax[1])

#ax.set_xlim([-1.5,1.5])
ax[1].set_ylabel('Conspiracy', labelpad=10)
ax[1].set(xlabel='News Media Conspumption')
#ax[1].set_title("Facebook and Twitter", fontsize=18)
#stats.pearsonr(df.Fauci, df.KNOWLEDGE_pre)

plt.legend(labels=['FOX', 'CNN', 'ABC', 'MSNBC', 'NBC','CBS','PBS'])
plt.savefig('news.tif', dpi=900, format="tiff")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

%load_ext rpy2.ipython
# %R library(lme4)

%R library(lmerTest)

In [ ]:
%Rpush DATA

In [ ]:
%%R

M <- lmer(Change ~ SpeakAgree + SelfAgree + (1|partnum) + (1|itemnum), data = DATA)
print(summary(M))

In [ ]:
%%R

M <- lmer(Change ~ SpeakAgree:lmh + (1 | partnum) + (1 | itemnum), data = DATA)
print(summary(M))

In [ ]:
aov = pg.mixed_anova(dv='Conv', within='Type', between='Epistemic', subject='Partnum', data=DAT)
pg.print_table(aov)

In [ ]:
pal = ["#DDA0DD", "#12a1eb"]

fig, ax = plt.subplots(1,3, figsize=(6,3.7), sharey=True)

sns.barplot(x=df.Epistemic, y=df.FACT_AGREE, data=df, \
            palette=pal, linewidth = 0, edgecolor=".2", errcolor=".2", \
            ax=ax[0])
sns.despine()
ax[0].set_xticklabels(["Low","High"])
ax[0].set_ylabel("Conversational endorsement")
ax[0].set(title = "Accurate")
bff.panellabel(ax[0], "A")

datahandles = np.array([L.get_xydata() for L in ax[0].get_lines()])
_, p = stats.ttest_ind(df.loc[df.Epistemic==100, "FACT_AGREE"], \
                       df.loc[df.Epistemic==200, "FACT_AGREE"])
bff.barplot_annotate_brackets(ax[0], 0, 1, p, datahandles, dh=0.05, fs=14)

sns.barplot(x=df.Epistemic, y=df.MISINFO_AGREE, data=df, \
            palette=pal, linewidth = 0, edgecolor=".2", errcolor=".2", \
            ax=ax[1])
sns.despine()
ax[1].set_xticklabels(["Low","High"])
ax[1].set_ylabel("", labelpad=8)
ax[1].set(title = "Inaccurate")
bff.panellabel(ax[1], "B")

datahandles = np.array([L.get_xydata() for L in ax[1].get_lines()])
_, p = stats.ttest_ind(df.loc[df.Epistemic==100, "MISINFO_AGREE"], \
                       df.loc[df.Epistemic==200, "MISINFO_AGREE"])
bff.barplot_annotate_brackets(ax[1], 0, 1, p, datahandles, dh=0.05, fs=14)

sns.barplot(x=df.Epistemic, y=df.CONSPIRACY_AGREE, data=df, \
            palette=pal, linewidth = 0, edgecolor=".2", errcolor=".2", \
            ax=ax[2])
sns.despine()
ax[2].set_xticklabels(["Low","High"])
ax[2].set_ylabel("", labelpad=8)
ax[2].set(title = "Conspiracy")
bff.panellabel(ax[2], "C")

datahandles = np.array([L.get_xydata() for L in ax[2].get_lines()])
_, p = stats.ttest_ind(df.loc[df.Epistemic==100, "CONSPIRACY_AGREE"], \
                       df.loc[df.Epistemic==200, "CONSPIRACY_AGREE"])
bff.barplot_annotate_brackets(ax[2], 0, 1, p, datahandles, dh=0.05, fs=14)

plt.ylim(-2.5,6.5)
plt.tight_layout()
plt.savefig('fig5.tif', dpi=900, format="tiff")

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(10,4), sharey=True)

sns.regplot(x=df.Trump, y=df.KNOWLEDGE_pre, scatter_kws={"color": "#848b79","alpha":.3}, \
            line_kws={"color":"#41A317","alpha":1,"lw":4}, ax=ax[0])
ax[0].set_ylabel('Knowledge (pre)')
ax[0].set(xlabel='Trust in Trump')
ax[0].set_title("Knowledge", fontsize=14)

sns.regplot(x=df.Fauci, y=df.KNOWLEDGE_pre, scatter_kws={"color": "#848b79","alpha":.3}, \
            line_kws={"color":"#41A317","alpha":1,"lw":4}, ax=ax[1])
ax[1].set_ylabel('Knowledge (pre)', labelpad=10)
ax[1].set(xlabel='Trust in Fauci')
ax[1].set_title("Knowledge", fontsize=14)


sns.regplot(x=df.Trump, y=df.CONSPIRACY_pre, scatter_kws={"color": "#C06C84","alpha":.3}, \
            line_kws={"color":"#7D0552","alpha":1,"lw":4}, ax=ax[2])
ax[2].set_ylabel('Conspiracy (pre)', labelpad=10)
ax[2].set(xlabel='Trust in Trump')
ax[2].set_title("Conspiracy", fontsize=14)


sns.regplot(x=df.Fauci, y=df.CONSPIRACY_pre, scatter_kws={"color": "#C06C84","alpha":.3}, \
            line_kws={"color":"#7D0552","alpha":1,"lw":4}, ax=ax[3])
ax[3].set_ylabel('Conspiracy (pre)', labelpad=10)
ax[3].set(xlabel='Trust in Fauci')
ax[3].set_title("Conspiracy", fontsize=14)

plt.ylim(-2.5,10.5)
plt.tight_layout()
plt.savefig('fig2.tif', dpi=900, format="tiff")

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(10,4), sharey=True)

sns.regplot(x=df.MEDIA, y=df.KNOWLEDGE_pre, scatter_kws={"color": "#848b79","alpha":.3}, \
            line_kws={"color":"#41A317","alpha":1,"lw":4}, ax=ax[0])
ax[0].set_ylabel('Knowledge (pre)')
ax[0].set(xlabel='News Media')
ax[0].set_title("Knowledge", fontsize=14)

sns.regplot(x=df.SocialMEDIA, y=df.KNOWLEDGE_pre, scatter_kws={"color": "#848b79","alpha":.3}, \
            line_kws={"color":"#41A317","alpha":1,"lw":4}, ax=ax[1])
ax[1].set_ylabel('Knowledge (pre)', labelpad=10)
ax[1].set(xlabel='Social Media')
ax[1].set_title("Knowledge", fontsize=14)


sns.regplot(x=df.MEDIA, y=df.CONSPIRACY_pre, scatter_kws={"color": "#C06C84","alpha":.3}, \
            line_kws={"color":"#7D0552","alpha":1,"lw":4}, ax=ax[2])
ax[2].set_ylabel('Conspiracy (pre)', labelpad=10)
ax[2].set(xlabel='News Media')
ax[2].set_title("Conspiracy", fontsize=14)


sns.regplot(x=df.SocialMEDIA, y=df.CONSPIRACY_pre, scatter_kws={"color": "#C06C84","alpha":.3}, \
            line_kws={"color":"#7D0552","alpha":1,"lw":4}, ax=ax[3])
ax[3].set_ylabel('Conspiracy (pre)', labelpad=10)
ax[3].set(xlabel='Social Media')
ax[3].set_title("Conspiracy", fontsize=14)

#plt.ylim(-2.5,10.5)
plt.tight_layout()
plt.savefig('fig3.tif', dpi=900, format="tiff")